In [1]:
!pip install peft
!pip install bitsandbytes
!pip install deepspeed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.1-py3-none-any.whl size=1483863 sha256=1b87b0dd06551f462144425c696b46bf06bfc3fe0189360d3effd07c6f393f1c
  Stored in directory: /root/.cache/pip/wheels/da/cb/14/9cbba50c73df044eb32a7ca29e34844c5f8959e12d22ae8b60
Successfully built deepspeed


In [2]:
# pip install accelerate
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import Trainer, BitsAndBytesConfig
import torch
import transformers
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    TaskType
)
torch.random.manual_seed(0)

In [3]:
!pip list

Package                          Version
-------------------------------- -------------------
absl-py                          1.4.0
accelerate                       0.34.2
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.16
albumentations                   1.4.15
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.19.0
astropy                          6.1.3
astropy-iers-data                0.2024.9.16.0.32.21
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread                        3.0.1
autograd                 

**모델 및 토크나이저 생성**

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",

)
model.enable_input_require_grads()
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

trainable params: 1,572,864 || all params: 3,822,652,416 || trainable%: 0.0411


In [5]:
import os
import numpy as np

In [6]:
os.getcwd()

'/content'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
train_file = open('/content/drive/MyDrive/AI_Practice/ratings_train.txt', 'r', encoding='utf-8')
test_file = open('/content/drive/MyDrive/AI_Practice/ratings_test.txt', 'r', encoding='utf-8')


In [9]:
lines = train_file.read().split('\n')
lines.pop(0)
lines.pop(-1)

train_sentences = []
train_labels = []

for line in lines:
  tks = line.split('\t')
  train_sentences.append(tks[1])
  train_labels.append(tks[2])

lines = test_file.read().split('\n')
lines.pop(0)
lines.pop(-1)

test_sentences = []
test_labels = []

for line in lines:
  tks = line.split('\t')
  test_sentences.append(tks[1])
  test_labels.append(tks[2])


In [10]:
total_num = 2000
max_length = 256
input_ids = np.zeros(shape=[total_num, max_length], dtype=np.int32)
attention_mask = np.zeros(shape=[total_num, max_length], dtype=np.int32)
labels = np.zeros(shape=[total_num, max_length], dtype=np.int32)

for i in range(total_num):
    instruction = f'''입력된 코멘트가 긍정인지 부정인지 예측하시오.

    input sentence:
    {train_sentences[i]}'''

    if train_labels[i] == '1':
      answer = '긍정적인 코멘트입니다.'
    else:
      answer = '부정적인 코멘트입니다.'

    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": f"{instruction}"}
    ]

    source_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="np"
    )[0]
    target_ids = tokenizer(answer + tokenizer.eos_token, add_special_tokens=False, return_tensors='np')['input_ids'][0]

    if len(source_ids) + len(target_ids) >= max_length:
        continue

    input_ids[i, :len(source_ids)] = source_ids[:]
    input_ids[i, len(source_ids):len(source_ids) + len(target_ids)] = target_ids[:]
    attention_mask[i, :len(source_ids) + len(target_ids)] = [1] * (len(source_ids) + len(target_ids))
    labels[i, :len(source_ids)] = [-100] * (len(source_ids))
    labels[i, len(source_ids):len(source_ids) + len(target_ids)] = target_ids[:]

np.save('input_ids', input_ids[:])
np.save('attention_mask', attention_mask[:])
np.save('labels', labels[:])


In [11]:
from torch.utils.data import Dataset
from dataclasses import dataclass, field
import transformers
from typing import Optional, Dict, Sequence, List

IGNORE_INDEX = -100


In [12]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        self.data_index = 0

        self.input_ids = np.load(f'input_ids.npy')
        self.attention_mask = np.load(f'attention_mask.npy')
        self.labels = np.load(f'labels.npy')

        self.i = 0

        print(len(self.input_ids))
        self.max_num = self.input_ids.shape[0]

    def __len__(self):
        return int(self.input_ids.shape[0])

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:

        input_ids = torch.tensor(self.input_ids[i], dtype=torch.long)
        attention_mask = torch.tensor(self.attention_mask[i], dtype=torch.long)
        labels = torch.tensor(self.labels[i], dtype=torch.long)
        labels[attention_mask == 0] = IGNORE_INDEX

        return dict(input_ids=input_ids, attention_mask=attention_mask, labels=labels)


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, attention_mask, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "attention_mask", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True,
                                                         padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=attention_mask,
        )


def make_supervised_data_module(tokenizer: transformers.PreTrainedTokenizer, data_args) -> Dict:
    """Make dataset and collator for supervised fine-tuning."""
    train_dataset = SupervisedDataset(tokenizer=tokenizer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    print('dataset length:', len(train_dataset))
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=512,
        metadata={"help": "Maximum sequence length. Sequences will be right padded (and possibly truncated)."},
    )



In [ ]:
training_args = TrainingArguments(
    'nsmc_phi',
    optim="paged_adamw_32bit",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8,
    logging_steps=1,
    save_strategy='steps',
    save_steps=500,
    num_train_epochs=2,
    learning_rate=3e-3,
    weight_decay=1e-1,
    warmup_steps=10,
    bf16=True,
    fp16=False,
    gradient_checkpointing=True,
    save_total_limit=3,
    log_level='debug',
)



data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=None)

trainer = Trainer(model=model, tokenizer=tokenizer, args=training_args, **data_module)
trainer.train()
model.save_pretrained(training_args.output_dir)


2000
dataset length: 2000


Using auto half precision backend
Currently training with a batch size of: 16


[2024-09-26 01:12:57,040] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


***** Running training *****
  Num examples = 2,000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 30
  Number of trainable parameters = 1,572,864
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
You are not running the flash-attention implementation, expect numerical differences.
/usr/local/lib/python3.10/dist-packages/torch/utils/ch

Step,Training Loss
1,1.882400
2,1.878500
3,1.818000
4,1.457800
5,0.725000
6,0.418400
7,0.333800
8,0.110700
9,0.086900
10,0.050400


In [ ]:
model = model.merge_and_unload()

In [ ]:
# 일부 데이터에 대해서만 평가(실행 시간을 줄이기 위함)
total_num = 200

cor = 0
count = 0

for i in range(total_num):
  instruction = f'''입력된 코멘트가 긍정인지 부정인지 예측하시오.

    input sentence:
    {test_sentences[i]}'''

  messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": f"{instruction}"}
    ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|endoftext|>")
  ]

  # generate에는 다양한 생성 옵션을 추가할 수 있음
  outputs = model.generate(
      input_ids,
      max_new_tokens=32,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  prediction = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

  if prediction.find('긍정') != -1:
    if test_labels[i] == '1':
      cor += 1
  if prediction.find('부정') != -1:
    if test_labels[i] == '0':
      cor += 1
  count += 1
  print(prediction)

print(cor / count)